In [27]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder
import sys
import os

In [47]:
import pandas as pd
import requests
from scipy.stats import entropy
from concurrent.futures import ThreadPoolExecutor, as_completed

def process_network_csv(csv_path, output_path=None):
    def get_country(ip):
        try:
            response = requests.get(f"https://ipapi.co/{ip}/country/", timeout=5)
            return response.text.strip()
        except:
            return "Unknown"

    def calculate_entropy(payload):
        return entropy(pd.Series(list(str(payload))).value_counts())

    # Read the CSV file and remove leading/trailing spaces from column names
    df = pd.read_csv(csv_path)
    df.columns = df.columns.str.strip()
    
    # Print the columns of the input CSV to verify the column names
    print("Columns in the input CSV file after stripping spaces:", df.columns)

    # Check for the required columns and handle missing ones
    required_columns = [
        'Flow Packets/s', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'SYN Flag Count', 'Protocol', 
        'Total Fwd Packets', 'Total Backward Packets', 'Flow ID', 'Source IP', 'Flow Duration', 'Flow IAT Mean', 
        'Timestamp', 'Fwd Packet Length Std', 'Bwd Packet Length Std', 'Packet Length Variance', 'Fwd IAT Std', 
        'Bwd IAT Std', 'Flow IAT Std', 'Active Mean', 'Active Std', 'Idle Mean', 'Idle Std', 
        'Subflow Fwd Packets', 'Subflow Bwd Packets', 'Subflow Fwd Bytes', 'Subflow Bwd Bytes', 
        'Init_Win_bytes_forward', 'Init_Win_bytes_backward', 'Fwd PSH Flags', 'Bwd PSH Flags', 
        'Fwd URG Flags', 'Bwd URG Flags'
    ]
    
    missing_columns = [col for col in required_columns if col not in df.columns]
    
    if missing_columns:
        raise KeyError(f"The following required columns are missing in the input CSV file: {', '.join(missing_columns)}")

    # 1. Traffic Volume Metrics
    df['Requests_per_Second'] = df['Flow Packets/s']
    df['Total_Bandwidth_Consumption'] = df['Total Length of Fwd Packets'] + df['Total Length of Bwd Packets']
    df['Packet_Rate'] = df['Flow Packets/s']

    # 2. Protocol-Specific Features
    df['TCP_SYN_Packet_Count'] = df['SYN Flag Count']
    df['UDP_Packet_Count'] = df[df['Protocol'] == 17]['Total Fwd Packets'] + df[df['Protocol'] == 17]['Total Backward Packets']
    df['ICMP_Packet_Count'] = df[df['Protocol'] == 1]['Total Fwd Packets'] + df[df['Protocol'] == 1]['Total Backward Packets']

    # 3. IP Address Diversity
    df['Unique_Source_IPs'] = df.groupby('Flow ID')['Source IP'].transform('nunique')

    # 4. Time-Based Features
    df['Connection_Duration'] = df['Flow Duration']
    df['Inter_Arrival_Time'] = df['Flow IAT Mean']
    df['Time_Pattern'] = pd.to_datetime(df['Timestamp']).dt.hour

    # 5. Payload Analysis
    df['Payload_Size'] = df['Total Length of Fwd Packets'] + df['Total Length of Bwd Packets']
    df['Payload_Entropy'] = (df['Total Length of Fwd Packets'] + df['Total Length of Bwd Packets']).apply(calculate_entropy)

    # 6. Flag Analysis
    flag_columns = ['FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count']
    df['Total_Flags'] = df[flag_columns].sum(axis=1)
    df['Flag_Distribution'] = df[flag_columns].apply(lambda row: ','.join(f"{col.split()[0]}:{val}" for col, val in row.items()), axis=1)

    # 7. Packet Length Statistics
    df['Fwd_Packet_Length_Std'] = df['Fwd Packet Length Std']
    df['Bwd_Packet_Length_Std'] = df['Bwd Packet Length Std']
    df['Packet_Length_Variance'] = df['Packet Length Variance']

    # 8. Flow Inter-arrival Time Statistics
    df['Fwd_IAT_Std'] = df['Fwd IAT Std']
    df['Bwd_IAT_Std'] = df['Bwd IAT Std']
    df['Flow_IAT_Std'] = df['Flow IAT Std']

    # 9. Active and Idle Time
    df['Active_Time_Mean'] = df['Active Mean']
    df['Active_Time_Std'] = df['Active Std']
    df['Idle_Time_Mean'] = df['Idle Mean']
    df['Idle_Time_Std'] = df['Idle Std']

    # 10. Subflow Statistics
    df['Subflow_Fwd_Packets'] = df['Subflow Fwd Packets']
    df['Subflow_Bwd_Packets'] = df['Subflow Bwd Packets']
    df['Subflow_Fwd_Bytes'] = df['Subflow Fwd Bytes']
    df['Subflow_Bwd_Bytes'] = df['Subflow Bwd Bytes']

    # 11. Window Size Statistics
    df['Init_Win_Bytes_Forward'] = df['Init_Win_bytes_forward']
    df['Init_Win_Bytes_Backward'] = df['Init_Win_bytes_backward']

    # 12. Packet Count Ratio
    df['Packet_Count_Ratio'] = df['Total Fwd Packets'] / (df['Total Backward Packets'] + 1)  # Adding 1 to avoid division by zero

    # 13. Byte Count Ratio
    df['Byte_Count_Ratio'] = df['Total Length of Fwd Packets'] / (df['Total Length of Bwd Packets'] + 1)  # Adding 1 to avoid division by zero

    # 14. PSH and URG Flag Ratios
    total_packets = df['Total Fwd Packets'] + df['Total Backward Packets']
    df['PSH_Flag_Ratio'] = (df['Fwd PSH Flags'] + df['Bwd PSH Flags']) / total_packets
    df['URG_Flag_Ratio'] = (df['Fwd URG Flags'] + df['Bwd URG Flags']) / total_packets

    # 15. Average Packet Size
    df['Avg_Packet_Size'] = (df['Total Length of Fwd Packets'] + df['Total Length of Bwd Packets']) / total_packets

    # Multithreading for geolocation API calls
    with ThreadPoolExecutor(max_workers=10) as executor:
        future_to_ip = {executor.submit(get_country, ip): ip for ip in df['Source IP'].unique()}
        for future in as_completed(future_to_ip):
            ip = future_to_ip[future]
            try:
                country = future.result()
                df.loc[df['Source IP'] == ip, 'Source_IP_Country'] = country
            except Exception as exc:
                print(f"{ip} generated an exception: {exc}")

    if output_path:
        df.to_csv(output_path, index=False)
        print(f"Enhanced network features have been generated and saved to '{output_path}'")
    
    return df




In [48]:
# Example usage: process the CSV and save the output
csv_path = r"C:\Users\kani2\Downloads\Datasets\GeneratedLabelledFlows (1)\Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv"
output_path = r'E:\LogicInsights\works\output.csv'

# Process the CSV and get the enhanced DataFrame
df_enhanced = process_network_csv(csv_path, output_path)

Columns in the input CSV file after stripping spaces: Index(['Flow ID', 'Source IP', 'Source Port', 'Destination IP',
       'Destination Port', 'Protocol', 'Timestamp', 'Flow Duration',
       'Total Fwd Packets', 'Total Backward Packets',
       'Total Length of Fwd Packets', 'Total Length of Bwd Packets',
       'Fwd Packet Length Max', 'Fwd Packet Length Min',
       'Fwd Packet Length Mean', 'Fwd Packet Length Std',
       'Bwd Packet Length Max', 'Bwd Packet Length Min',
       'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s',
       'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max',
       'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std',
       'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean',
       'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags',
       'Bwd PSH Flags', 'Fwd URG Flags', 'Bwd URG Flags', 'Fwd Header Length',
       'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s',
       'Min Packet 